**Problem 3: Sentiment Analysis**

Snappfood (an online food delivery company) user comments containing 70,000 comments with two labels: 1- Happy (Positive) 2- Sad (Negative). We want you to build naive bayes Classifier from scratch to perform sentiment analysis. (This link1 can be very useful). Follow the steps below:

In [ ]:
!pip install hazm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


a) Before building the model, you must do the required text preprocessing. Explain all pre-processing steps. (You can use the hazm library, which is used to process the Persian language. (Note that this section has the highest score).

In [ ]:
import csv
from hazm import *

import re


normalizer = Normalizer()
lemmatizer = Lemmatizer()
tokenizer = WordTokenizer()

def preprocess(text):
    
    # Remove all punctuations except for dot and question mark
    text = re.sub(r'[^\w\s؟.]', '', text)
    
    # Normalize text
    text = normalizer.normalize(text)
    
    # Tokenize text
    words = tokenizer.tokenize(text)
    
    # Lemmatize words and remove stop words
    words = [lemmatizer.lemmatize(word) for word in words if not word in stopwords_list()]
    
    # Join the words back into a string
    text = ' '.join(words)
    
    return text

In [ ]:
import tqdm

In [ ]:
import pandas as pd
from hazm import word_tokenize
from hazm import Normalizer
from hazm import stopwords_list
from hazm import Stemmer

df = pd.read_csv('Snappfood - Sentiment Analysis.csv', delimiter='\t')

#Remove unnecessary column

df = df.drop('label_id', axis=1)

# remove the first column which is empty
df.drop(df.columns[0], axis=1, inplace=True)

#Convert the labels to numerical values:

label_to_id = {'HAPPY': 0, 'SAD': 1}
df['label'] = df['label'].map(label_to_id)

## Normalize the tokens using the hazm library

# Apply the normalization function to the comments column
normalizer = Normalizer()
df['comment'] = df['comment'].apply(normalizer.normalize)
print(df['comment'].head())

#Tokenize the comments using the hazm library
print(df['comment'].head())
df['comment'] = df['comment'].apply(word_tokenize)
print(df['comment'].head())

#Remove stop words using the hazm library

stopwords = stopwords_list()
df['comment'] = df['comment'].apply(lambda tokens: [token for token in tokens if token not in stopwords])
print(df['comment'].head())
#Stem the tokens using the hazm library

stemmer = Stemmer()
df['comment'] = df['comment'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

print(df['comment'].head(10))

0      واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1    قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...
2    قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...
3    عالللی بود همه چه درست و به اندازه و کیفیت خوب...
4                        شیرینی وانیلی فقط یک مدل بود.
Name: comment, dtype: object
0      واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1    قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...
2    قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...
3    عالللی بود همه چه درست و به اندازه و کیفیت خوب...
4                        شیرینی وانیلی فقط یک مدل بود.
Name: comment, dtype: object
0    [واقعا, حیف, وقت, که, بنویسم, سرویس, دهیتون, ش...
1    [قرار, بود, ۱, ساعته, برسه, ولی, نیم, ساعت, زو...
2    [قیمت, این, مدل, اصلا, با, کیفیتش, سازگاری, ند...
3    [عالللی, بود, همه, چه, درست, و, به, اندازه, و,...
4               [شیرینی, وانیلی, فقط, یک, مدل, بود, .]
Name: comment, dtype: object
0     [واقعا, حیف, وقت, بنویسم, سرویس, دهیتون, افتضاح]
1    [قرار, ۱, ساعته, برسه, نیم, 

b) Building the naive bayes classifier. Explain how naive bayes is used for this problem.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Convert the list of tokenized words in 'comment' column to strings
df['comment'] = df['comment'].apply(lambda x: ' '.join(x))
print(df['comment'].head(10))
print(df.isnull().sum())
df.dropna(inplace=True)

# Convert preprocessed comments to numerical representation using TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['comment'])

# Split the dataset into training and testing sets
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = df['label'][:train_size], df['label'][train_size:]

# Train the Naive Bayes classifier
clf = MultinomialNB()

0               واقعا حیف وق بنویس سرویس دهیتون افتضاح
1    قرار ۱ ساعته برسه ن ساع زود موقع ، ببین چقدررر...
2    قیم مدل اصلا کیفیت سازگار نداره ، ظاهر فریبنده...
3    عاللل درس اندازه کیف ، امیداور کیفیتتون باشه م...
4                                    شیرین وانیل مدل .
5                                   بد پیتزا خورده_بود
6                                                ممنون
7    کیف غذا متوسط پایین انگار داخل یه رستور معمول ...
8           اقلا تازه روز وخیلییی سریع بدس واقعا متشکر
9               چ ه داگ دور کلا سوخته_بود داخل خا !!!!
Name: comment, dtype: object
comment      0
label      520
dtype: int64


c) Fitting the model on training set and evaluating accuracies on the test set.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support



#fit classifier

clf.fit(X_train, y_train)

# Test the classifier
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate precision, recall, and F1-score
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred, average='weighted')

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.80
Confusion Matrix:
[[5080 1921]
 [ 839 6056]]
Precision: 0.8090957260141125
Recall: 0.8013816925734024
F1-score: 0.8002900902166333
